# Instalisasi packages firebase dan pyspark

In [ ]:
pip install requests

In [ ]:
pip install python-firebase

In [ ]:
!wget https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar -xvzf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark

--2021-01-09 08:49:36--  https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220272364 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.0-bin-hadoop2.7.tgz.3’

spark-3.0.0-bin-had 100%[===================>] 210.07M  32.0MB/s    in 6.5s    

2021-01-09 08:49:42 (32.5 MB/s) - ‘spark-3.0.0-bin-hadoop2.7.tgz.3’ saved [220272364/220272364]

spark-3.0.0-bin-hadoop2.7/
spark-3.0.0-bin-hadoop2.7/NOTICE
spark-3.0.0-bin-hadoop2.7/kubernetes/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.0-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.0-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.0-bin-hado

import packages dan library yang diperlukan

In [ ]:
import pandas as pd
import numpy as np
import os
import findspark
from firebase import firebase
import xml.etree.ElementTree as et 

In [ ]:

os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

findspark.init()


In [ ]:


from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Membuat dataframe covid19").getOrCreate()
print(spark.sparkContext.appName)


Membuat dataframe covid19


#Koneksi Google Drive pada Runtime

In [ ]:
#Mount google drive dengan akun pribadi
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#set path untuk datasetnya pada google drive yang sudah di mount pada runtime
DATASET_PATH = '/content/drive/MyDrive/dataset_bigdata'

# Membandingkan Ketiga Dataframe

In [ ]:
def parse_XML(xml_file, df_cols): 
  """Parse the input XML file and store the result in a pandas 
  DataFrame with the given columns. 

  The first element of df_cols is supposed to be the identifier 
  variable, which is an attribute of each node element in the 
  XML data; other features will be parsed from the text content 
  of each sub-element. 
  """

  xtree = et.parse(xml_file)
  xroot = xtree.getroot()
  rows = []

  for node in xroot: 
    res = []
    res.append(node.attrib.get(df_cols[0]))
    for el in df_cols[1:]: 
      if node is not None and node.find(el) is not None:
        res.append(node.find(el).text)
      else: 
        res.append(None)
    rows.append({df_cols[i]: res[i] for i, _ in enumerate(df_cols)})

  out_df = pd.DataFrame(rows, columns=df_cols)
      
  return out_df

In [ ]:
pathAr = DATASET_PATH+"/saudiarabia.csv"

dtAr = spark.read.csv(pathAr, sep=",", header=True)

dtAr.show()

+------------+-----------+--------+----+--------+-----+-----+-----+---------+--------------------+
|     Country|CountryCode|Province|City|CityCode|  Lat|  Lon|Cases|   Status|                Date|
+------------+-----------+--------+----+--------+-----+-----+-----+---------+--------------------+
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    1|confirmed|2020-03-02T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    1|confirmed|2020-03-03T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    1|confirmed|2020-03-04T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    5|confirmed|2020-03-05T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    5|confirmed|2020-03-06T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    5|confirmed|2020-03-07T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|   11|confirmed|2020-03-08T00:00:00Z|
|Saudi Ara

In [ ]:
dt_ind = pd.read_json(DATASET_PATH+"/indonesia.json")
display(dt_ind)

,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date
0,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-02 00:00:00+00:00
1,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-03 00:00:00+00:00
2,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-04 00:00:00+00:00
3,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-05 00:00:00+00:00
4,Indonesia,ID,,,,-0.79,113.92,4,0,0,4,2020-03-06 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
238,Indonesia,ID,,,,-0.79,113.92,392934,13411,317672,61851,2020-10-26 00:00:00+00:00
239,Indonesia,ID,,,,-0.79,113.92,396454,13512,322248,60694,2020-10-27 00:00:00+00:00
240,Indonesia,ID,,,,-0.79,113.92,400483,13612,325793,61078,2020-10-28 00:00:00+00:00
241,Indonesia,ID,,,,-0.79,113.92,404048,13701,329778,60569,2020-10-29 00:00:00+00:00


In [ ]:
df_zimbabwe = parse_XML(DATASET_PATH+"/zimbabwe.xml", ["Active", "Confirmed", "Country", "Code", "Date","Deaths","Latitude","Longitude","Recovered"])
display(df_zimbabwe)

,Active,Confirmed,Country,Code,Date,Deaths,Latitude,Longitude,Recovered
0,None,1,Zimbabwe,ZW,2020-03-20,0,-19.02,29.15,0
1,None,3,Zimbabwe,ZW,2020-03-21,0,-19.02,29.15,0
2,None,3,Zimbabwe,ZW,2020-03-22,0,-19.02,29.15,0
3,None,3,Zimbabwe,ZW,2020-03-23,1,-19.02,29.15,0
4,None,3,Zimbabwe,ZW,2020-03-24,1,-19.02,29.15,0
...,...,...,...,...,...,...,...,...,...
220,None,8303,Zimbabwe,ZW,2020-10-26,242,-19.02,29.15,7797
221,None,8315,Zimbabwe,ZW,2020-10-27,242,-19.02,29.15,7804
222,None,8320,Zimbabwe,ZW,2020-10-28,242,-19.02,29.15,7845
223,None,8349,Zimbabwe,ZW,2020-10-29,242,-19.02,29.15,7864


In [ ]:
print("Arab: ", dtAr.columns, "\n","Indonesia: ", dt_ind.columns, "\n", "Zimbabwe: ", df_zimbabwe.columns)

Arab:  ['Country', 'CountryCode', 'Province', 'City', 'CityCode', 'Lat', 'Lon', 'Cases', 'Status', 'Date'] 
 Indonesia:  Index(['Country', 'CountryCode', 'Province', 'City', 'CityCode', 'Lat', 'Lon',
       'Confirmed', 'Deaths', 'Recovered', 'Active', 'Date'],
      dtype='object') 
 Zimbabwe:  Index(['Active', 'Confirmed', 'Country', 'Code', 'Date', 'Deaths', 'Latitude',
       'Longitude', 'Recovered'],
      dtype='object')


Dari ketiga Dataframe tersebut, didapat bahwa kolom yang beririsan yaitu: **Country, Countrycode, Lat, Lon, Cases/Confirmed, dan Date**

## Melakukan cleanse data Negara Saudi Arabia

In [ ]:
dtAr.show()

+------------+-----------+--------+----+--------+-----+-----+-----+---------+--------------------+
|     Country|CountryCode|Province|City|CityCode|  Lat|  Lon|Cases|   Status|                Date|
+------------+-----------+--------+----+--------+-----+-----+-----+---------+--------------------+
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    1|confirmed|2020-03-02T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    1|confirmed|2020-03-03T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    1|confirmed|2020-03-04T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    5|confirmed|2020-03-05T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    5|confirmed|2020-03-06T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|    5|confirmed|2020-03-07T00:00:00Z|
|Saudi Arabia|         SA|    null|null|    null|23.89|45.08|   11|confirmed|2020-03-08T00:00:00Z|
|Saudi Ara

In [ ]:
dtArab = dtAr.drop('Province','City','CityCode','Status')
dtArab.columns

['Country', 'CountryCode', 'Lat', 'Lon', 'Cases', 'Date']

## Melakukan cleanse data Negara Indonesia

In [ ]:
display(dt_ind)

,Country,CountryCode,Province,City,CityCode,Lat,Lon,Confirmed,Deaths,Recovered,Active,Date
0,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-02 00:00:00+00:00
1,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-03 00:00:00+00:00
2,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-04 00:00:00+00:00
3,Indonesia,ID,,,,-0.79,113.92,2,0,0,2,2020-03-05 00:00:00+00:00
4,Indonesia,ID,,,,-0.79,113.92,4,0,0,4,2020-03-06 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
238,Indonesia,ID,,,,-0.79,113.92,392934,13411,317672,61851,2020-10-26 00:00:00+00:00
239,Indonesia,ID,,,,-0.79,113.92,396454,13512,322248,60694,2020-10-27 00:00:00+00:00
240,Indonesia,ID,,,,-0.79,113.92,400483,13612,325793,61078,2020-10-28 00:00:00+00:00
241,Indonesia,ID,,,,-0.79,113.92,404048,13701,329778,60569,2020-10-29 00:00:00+00:00


In [ ]:
del dt_ind['Recovered']
del dt_ind['Active']
del dt_ind['Deaths']
del dt_ind['Province']
del dt_ind['City']
del dt_ind['CityCode']
dt_indo = dt_ind.rename(columns = {"Confirmed":"Cases"}) 
dt_ind.columns



## Melakukan cleanse data Negara Zimbabwe

In [ ]:
display(df_zimbabwe)

,Active,Confirmed,Country,Code,Date,Deaths,Latitude,Longitude,Recovered
0,None,1,Zimbabwe,ZW,2020-03-20,0,-19.02,29.15,0
1,None,3,Zimbabwe,ZW,2020-03-21,0,-19.02,29.15,0
2,None,3,Zimbabwe,ZW,2020-03-22,0,-19.02,29.15,0
3,None,3,Zimbabwe,ZW,2020-03-23,1,-19.02,29.15,0
4,None,3,Zimbabwe,ZW,2020-03-24,1,-19.02,29.15,0
...,...,...,...,...,...,...,...,...,...
220,None,8303,Zimbabwe,ZW,2020-10-26,242,-19.02,29.15,7797
221,None,8315,Zimbabwe,ZW,2020-10-27,242,-19.02,29.15,7804
222,None,8320,Zimbabwe,ZW,2020-10-28,242,-19.02,29.15,7845
223,None,8349,Zimbabwe,ZW,2020-10-29,242,-19.02,29.15,7864


In [ ]:
del df_zimbabwe['Recovered']
del df_zimbabwe['Active']
del df_zimbabwe['Deaths']
dt_zimbabwe = df_zimbabwe.rename(columns = {"Confirmed":"Cases", "Code":"CountryCode", "Latitude":"Lat", "Longitude":"Lon"}) 
dt_zimbabwe = dt_zimbabwe[['Country', 'CountryCode', "Lat", "Lon", "Cases", "Date"]]
dt_zimbabwe.columns

Index(['Country', 'CountryCode', 'Lat', 'Lon', 'Cases', 'Date'], dtype='object')

# Config Realtime Database Firebase agar terhubung dengan runtime

In [ ]:
firebase = firebase.FirebaseApplication('https://big-data-project-2d675-default-rtdb.firebaseio.com/', None)

## Memasukkan data Negara Arab ke Firebase

In [ ]:
dtArab.show()

+------------+-----------+-----+-----+-----+--------------------+
|     Country|CountryCode|  Lat|  Lon|Cases|                Date|
+------------+-----------+-----+-----+-----+--------------------+
|Saudi Arabia|         SA|23.89|45.08|    1|2020-03-02T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|    1|2020-03-03T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|    1|2020-03-04T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|    5|2020-03-05T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|    5|2020-03-06T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|    5|2020-03-07T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|   11|2020-03-08T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|   15|2020-03-09T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|   20|2020-03-10T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|   21|2020-03-11T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|   45|2020-03-12T00:00:00Z|
|Saudi Arabia|         SA|23.89|45.08|   86|2020-03-13T00:00:00Z|
|Saudi Ara

konversi dataframe pySpark menjadi pandas

In [ ]:
dfArabs = dtArab.toPandas()
dfArabs

,Country,CountryCode,Lat,Lon,Cases,Date
0,Saudi Arabia,SA,23.89,45.08,1,2020-03-02T00:00:00Z
1,Saudi Arabia,SA,23.89,45.08,1,2020-03-03T00:00:00Z
2,Saudi Arabia,SA,23.89,45.08,1,2020-03-04T00:00:00Z
3,Saudi Arabia,SA,23.89,45.08,5,2020-03-05T00:00:00Z
4,Saudi Arabia,SA,23.89,45.08,5,2020-03-06T00:00:00Z
...,...,...,...,...,...,...
238,Saudi Arabia,SA,23.89,45.08,345232,2020-10-26T00:00:00Z
239,Saudi Arabia,SA,23.89,45.08,345631,2020-10-27T00:00:00Z
240,Saudi Arabia,SA,23.89,45.08,346047,2020-10-28T00:00:00Z
241,Saudi Arabia,SA,23.89,45.08,346482,2020-10-29T00:00:00Z


In [ ]:
#Menghapus data di firebase terlebih dahulu jika sudah ada
firebase.delete('/SaudiArab/', '')
print('Record Deleted')

Record Deleted


Melakukan perulangan agar yang masuk ke firebase merupakan data per rowsnya

In [ ]:
for index, row in dfArabs.iterrows():
  data = {
      'Country': row['Country'], 
      'CountryCode': row['CountryCode'], 
      'Lat':  row['Lat'], 
      'Lon':  row['Lon'], 
      'Cases':  row['Cases'], 
      'Date':  row['Date']
      }
  result = firebase.post('/SaudiArab/',data)
  print(row['Cases'], row['Date'])


1 2020-03-02T00:00:00Z
1 2020-03-03T00:00:00Z
1 2020-03-04T00:00:00Z
5 2020-03-05T00:00:00Z
5 2020-03-06T00:00:00Z
5 2020-03-07T00:00:00Z
11 2020-03-08T00:00:00Z
15 2020-03-09T00:00:00Z
20 2020-03-10T00:00:00Z
21 2020-03-11T00:00:00Z
45 2020-03-12T00:00:00Z
86 2020-03-13T00:00:00Z
103 2020-03-14T00:00:00Z
103 2020-03-15T00:00:00Z
118 2020-03-16T00:00:00Z
171 2020-03-17T00:00:00Z
171 2020-03-18T00:00:00Z
274 2020-03-19T00:00:00Z
344 2020-03-20T00:00:00Z
392 2020-03-21T00:00:00Z
511 2020-03-22T00:00:00Z
562 2020-03-23T00:00:00Z
767 2020-03-24T00:00:00Z
900 2020-03-25T00:00:00Z
1012 2020-03-26T00:00:00Z
1104 2020-03-27T00:00:00Z
1203 2020-03-28T00:00:00Z
1299 2020-03-29T00:00:00Z
1453 2020-03-30T00:00:00Z
1563 2020-03-31T00:00:00Z
1720 2020-04-01T00:00:00Z
1885 2020-04-02T00:00:00Z
2039 2020-04-03T00:00:00Z
2179 2020-04-04T00:00:00Z
2402 2020-04-05T00:00:00Z
2605 2020-04-06T00:00:00Z
2795 2020-04-07T00:00:00Z
2932 2020-04-08T00:00:00Z
3287 2020-04-09T00:00:00Z
3651 2020-04-10T00:00:00Z
40

## Memasukkan data Negara Indonesia ke Firebase

In [ ]:
display(dt_indo)

In [ ]:
#Menghapus data di firebase terlebih dahulu jika sudah ada
firebase.delete('/Indonesia/', '')
print('Record Deleted')

Melakukan perulangan agar yang masuk ke firebase merupakan data per rowsnya

In [ ]:
for index, row in dt_indo.iterrows():
  data = {
      'Country': row['Country'], 
      'CountryCode': row['CountryCode'], 
      'Lat':  row['Lat'], 
      'Lon':  row['Lon'], 
      'Cases':  row['Cases'], 
      'Date':  row['Date']
      }
  result = firebase.post('/Indonesia/',data)
  print(row['Cases'], row['Date'])

## Memasukkan data Negara Zimbabwe ke Firebase

In [ ]:
display(dt_zimbabwe)

Melakukan perulangan agar yang masuk ke firebase merupakan data per rowsnya

In [ ]:
#Menghapus data di firebase terlebih dahulu jika sudah ada
firebase.delete('/Zimbabwe/', '')
print('Record Deleted')

In [ ]:
for index, row in dt_zimbabwe.iterrows():
  data = {
      'Country': row['Country'], 
      'CountryCode': row['CountryCode'], 
      'Lat':  row['Lat'], 
      'Lon':  row['Lon'], 
      'Cases':  row['Cases'], 
      'Date':  row['Date']
      }
  result = firebase.post('/Zimbabwe/',data)
  print(row['Cases'], row['Date'])

## Cek hasil dari firebase setiap negara

In [ ]:
resAr = firebase.get('/SaudiArab/', '')
resId = firebase.get('/Indonesia/', '')
resZw = firebase.get('/Zimbabwe/', '')
print("Arab: ", resAr, "\n Indonesia: ", resId, "\n Zimbabwe: ", resZw)

### Saudi Arabia
Melakukan konversi dari dictionary menjadi dataframe

In [ ]:
 dfFbAr = pd.DataFrame.from_dict(resAr)
 display(dfFbAr)

### Indonesia
Melakukan konversi dari dictionary menjadi dataframe

In [ ]:
dfFbId = pd.DataFrame.from_dict(resId)
display(dfFbId)

### Zimbabwe
Melakukan konversi dari dictionary menjadi dataframe

In [ ]:
dfFbZw = pd.DataFrame.from_dict(resZw)
display(dfFbZw)